In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sklearn
from chinese_calendar import is_workday

In [2]:
# 先製造日期在計算工作日，接著在和原本資料合併

In [3]:
def work_day(data,year):
    check_date = pd.read_csv(f"./週期資料.csv",encoding='utf-8')
    answer =  pd.DataFrame()
    date =  pd.DataFrame()
    date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
    date['週'] = date['日期'].dt.isocalendar().week
    date['年'] = date['日期'].dt.isocalendar().year
    date = date.drop(date[date['週'] == 53].index).reset_index()
    del_data =  date[date['週'] == 52].tail(1).index
    date = date.drop(date[del_data.values[0]+1:].index).reset_index()
    date["日期"] = pd.to_datetime(date["日期"] ,format='%Y/%m/%d')
    check_date["date"] = pd.to_datetime(check_date["date"] ,format='%Y/%m/%d')
    # 如果是工作日就存成true，否則存成false
    date['休假日']=date['日期'].map(lambda x:(check_date['date']==x).any())
    print(date)

    for num in range(1, 53,1):
        temp = date[date['週'] == num]
#         計算該周工作日
        holiday = len(temp[temp['休假日'] == True])
        work_day = len(temp[temp['休假日'] == False])
        date.loc[date['週']==num,'工作天數'] = work_day
        date.loc[date['週']==num,'休假天數'] = holiday
        if year == 2021 and num == 52:   
            date.loc[date['週']==num,'休假天數'] = 2

#     同一週期工作日、休假日都一樣所以只保留一筆
    date.drop_duplicates(subset='週', keep='last', inplace=True)
    date = date.drop(['日期', '休假日','level_0','index'], axis=1)
    print(date)
    answer = pd.merge(data, date, on='週',how='outer')
#     刪除日期和工作日

    
    return answer

資料前處理

In [4]:
def data_process(data):
    data = data.reset_index(drop=True)
    for i in range(1, 52,1):
#         判斷是否每一週都有值如果沒有補值
        have_value = data[data['週'] == i]
#         如果是空值
        if have_value.empty == True:
            temp = data.loc[0:0]
            temp['受料量(+)'] = 0
            temp['週'] = i 
            data = data.append(temp,ignore_index=True)
#     依照週排序
    data = data.sort_values(by='週').reset_index(drop=True)
#     print(data)
    return data

讀取資料+整理工作日資料

In [5]:
# 料號
# F10110303.csv  50
# B03251164.csv  50
# B07109004.csv  50
# B05569068.csv  49

part_no = 'B05569068'
# 病房
room = '五病房'
# 年分
years = [2019,2020,2021]

total_data =  pd.DataFrame()

for year in years:
    data = pd.read_csv(f"../使用量計算/沒有紀錄的衛材領用量/{room}/全部/{part_no}.csv",encoding='utf_8_sig')
    data = data[data.columns.drop(list(data.filter(regex='Unnamed')))].reset_index(drop=True)
#     加入年的欄位
    data["日期"] = pd.to_datetime(data["日期"] ,format='%Y/%m/%d')
    data['年'] = data['日期'].dt.isocalendar().year
    temp = data[data['年'] == year]
    data = data_process(temp)

    # 先刪除大於52周的資料
    del_data =  data[data['週'] == 52].tail(1).index
    if del_data.empty == False:
        data = data.drop(data[del_data.values[0]+1:].index).reset_index()
    print("測試")
    print(data)
    # 計算工作日
    data = work_day(data,year)
    total_data = pd.concat([total_data, data])
    #     刪除多的欄位
total_data = total_data[total_data.columns.drop(list(total_data.filter(regex='index')))].reset_index(drop=True)
total_data = total_data[total_data.columns.drop(list(total_data.filter(regex='level_0')))].reset_index(drop=True)
# # 把相同的值補上
# temp = data.head(1)
# total_data['庫房'] = temp['庫房'].values[0]
# total_data['料號'] = temp['料號'].values[0]
# total_data['品名'] = temp['品名'].values[0]
# total_data['數量'].fillna(0,inplace=True)

total_data = total_data[total_data.columns.drop(list(data.filter(regex='年_y')))]
total_data.rename(columns={'年_x': '年'}, inplace=True)
total_data.rename(columns={'受料量(+)': '數量'}, inplace=True)
total_data.rename(columns={'日期': '帳務日期'}, inplace=True)
total_data.to_csv(f"./領用量有加工作日/{room}/{part_no}.csv",encoding='utf_8_sig')
print(total_data)

    

C:\Users\IDSL\AppData\Local\Temp\ipykernel_22068\3929158266.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['受料量(+)'] = 0
C:\Users\IDSL\AppData\Local\Temp\ipykernel_22068\3929158266.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['週'] = i
C:\Users\IDSL\AppData\Local\Temp\ipykernel_22068\3929158266.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(temp,ignore_index=True)
C:\Users\

測試
    index         日期         料號       品名    規格 單位 屬性    常備量  昨日庫存量  批價量(-)  \
0       0 2019-01-08  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   
1       1 2019-01-08  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   
2       2 2019-01-15  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   
3       3 2019-01-22  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   
4       4 2019-01-29  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   
5       5 2019-02-01  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   
6       6 2019-01-08  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   
7       7 2019-02-12  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   
8       8 2019-02-19  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   
9       9 2019-02-27  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   
10     10 2019-03-05  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   
11     11 2019-03-14  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0  

C:\Users\IDSL\AppData\Local\Temp\ipykernel_22068\4246247627.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
C:\Users\IDSL\AppData\Local\Temp\ipykernel_22068\3929158266.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['受料量(+)'] = 0
C:\Users\IDSL\AppData\Local\Temp\ipykernel_22068\3929158266.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

     level_0  index         日期   週     年    休假日
0          0      0 2020-01-01   1  2020   True
1          1      1 2020-01-02   1  2020  False
2          2      2 2020-01-03   1  2020  False
3          3      3 2020-01-04   1  2020   True
4          4      4 2020-01-05   1  2020   True
..       ...    ...        ...  ..   ...    ...
357      357    357 2020-12-23  52  2020  False
358      358    358 2020-12-24  52  2020  False
359      359    359 2020-12-25  52  2020  False
360      360    360 2020-12-26  52  2020   True
361      361    361 2020-12-27  52  2020   True

[362 rows x 6 columns]
      週     年  工作天數  休假天數
4     1  2020   2.0   3.0
11    2  2020   5.0   2.0
18    3  2020   5.0   2.0
25    4  2020   3.0   4.0
32    5  2020   2.0   5.0
39    6  2020   5.0   2.0
46    7  2020   5.0   2.0
53    8  2020   5.0   2.0
60    9  2020   4.0   3.0
67   10  2020   5.0   2.0
74   11  2020   5.0   2.0
81   12  2020   5.0   2.0
88   13  2020   5.0   2.0
95   14  2020   3.0   4.0
102  15  2

          帳務日期         料號       品名    規格 單位 屬性    常備量  昨日庫存量  批價量(-)  領用量(-)  \
0   2019-01-08  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   350.0   
1   2019-01-08  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   350.0   
2   2019-01-15  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   100.0   
3   2019-01-22  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   200.0   
4   2019-01-29  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   150.0   
..         ...        ...      ...   ... .. ..    ...    ...     ...     ...   
159 2021-11-30  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   100.0   
160 2021-12-07  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   200.0   
161 2021-12-16  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   150.0   
162 2021-12-22  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   200.0   
163 2021-12-28  B05569068  沖洗棉棒-6支  6支/包  包  T  300.0    0.0     0.0   300.0   

        數量  本日庫存量  退貨量(-)   週     年  工作

In [6]:
total_data.head(53)

,帳務日期,料號,品名,規格,單位,屬性,常備量,昨日庫存量,批價量(-),領用量(-),數量,本日庫存量,退貨量(-),週,年,工作天數,休假天數
0,2019-01-08,B05569068,沖洗棉棒-6支,6支/包,包,T,300.0,0.0,0.0,350.0,0.0,0.0,NaN,1,2019,3.0,3.0
1,2019-01-08,B05569068,沖洗棉棒-6支,6支/包,包,T,300.0,0.0,0.0,350.0,350.0,0.0,NaN,2,2019,5.0,2.0
2,2019-01-15,B05569068,沖洗棉棒-6支,6支/包,包,T,300.0,0.0,0.0,100.0,100.0,0.0,NaN,3,2019,5.0,2.0
3,2019-01-22,B05569068,沖洗棉棒-6支,6支/包,包,T,300.0,0.0,0.0,200.0,200.0,0.0,NaN,4,2019,5.0,2.0
4,2019-01-29,B05569068,沖洗棉棒-6支,6支/包,包,T,300.0,0.0,0.0,150.0,150.0,0.0,NaN,5,2019,5.0,2.0
5,2019-02-01,B05569068,沖洗棉棒-6支,6支/包,包,T,300.0,0.0,0.0,150.0,150.0,0.0,NaN,5,2019,5.0,2.0
6,2019-01-08,B05569068,沖洗棉棒-6支,6支/包,包,T,300.0,0.0,0.0,350.0,0.0,0.0,NaN,6,2019,0.0,7.0
7,2019-02-12,B05569068,沖洗棉棒-6支,6支/包,包,T,300.0,0.0,0.0,150.0,150.0,0.0,NaN,7,2019,5.0,2.0
8,2019-02-19,B05569068,沖洗棉棒-6支,6支/包,包,T,300.0,0.0,0.0,150.0,150.0,0.0,NaN,8,2019,5.0,2.0
9,2019-02-27,B05569068,沖洗棉棒-6支,6支/包,包,T,300.0,0.0,0.0,200.0,200.0,0.0,NaN,9,2019,3.0,4.0


In [7]:
#抓取日歷國定假日
import requests

page = 0
isHoliday = pd.DataFrame()
while True:
    url = f"https://data.ntpc.gov.tw/api/datasets/308DCD75-6434-45BC-A95F-584DA4FED251/json?page={page}&size=1000"
    res = requests.get(url)
    resJson = res.json()
    resJson2 = pd.DataFrame(resJson)
    isHoliday = pd.concat([isHoliday, resJson2])
    page+=1
    if resJson[-1]['date']>'2022':
        break

In [8]:
isHoliday

,date,chinese,isholiday,holidaycategory,description
0,2013/1/1,...,是,放假之紀念日及節日,全國各機關學校放假一日。
1,2013/1/5,,是,星期六、星期日,
2,2013/1/6,,是,星期六、星期日,
3,2013/1/12,,是,星期六、星期日,
4,2013/1/13,,是,星期六、星期日,
...,...,...,...,...,...
310,2023/12/17,,是,星期六、星期日,
311,2023/12/23,,是,星期六、星期日,
312,2023/12/24,,是,星期六、星期日,
313,2023/12/30,,是,星期六、星期日,


In [9]:
isHoliday
isHoliday['date'] = pd.to_datetime(isHoliday['date'])
isHoliday['isHoliday']=isHoliday.apply(lambda row: 1 if row['isHoliday']=='是' else 0, axis=1)
#1為國定假日0則不是
isHoliday = isHoliday[['date','isHoliday']]
# isHoliday['Date'] = pd.to_datetime(isHoliday['Date'])
isHoliday = isHoliday[(isHoliday['date']<'2022')&(isHoliday['date']>='2019')]

isHoliday['週'] = isHoliday['date'].dt.isocalendar().week
# isHoliday['年'] = isHoliday['date'].dt.isocalendar().year
# isHoliday['月'] = isHoliday['date'].dt.month
# isHoliday[(isHoliday['年'] == 2021)&(isHoliday['月'] == 2)]
# isHoliday.to_csv(f"./週期資料.csv")

KeyError: 'isHoliday'

In [ ]:
total_data.tail(1500)